# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from collections import Counter
from matplotlib_venn import venn2, venn3
from scipy import stats
UKBB_PATH = '../glaucoma_project/UKBB_Data/'
UKBB_GENOME_PATH = '../glaucoma_project/UKBB_Data_Basket2/raw_genome/merged_raw_genome/'
UKBB_IMPUTED_GENOME_PATH = '../glaucoma_project/UKBB_Data_Basket2/imputed_genome/merged_imputed_genome/'
GWAS_PATH = '../glaucoma_project/GWAS/'
from sklearn.model_selection import train_test_split
from tableone import TableOne
import statsmodels.api as sm
import re
import seaborn as sns
import gzip
import time
import venn
tqdm.pandas()
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import shap
from joblib import dump, load

UKBB_PATH = '../glaucoma_project/UKBB_Data/'
MAIN_UKBB_FILE = '../glaucoma_project/UKBB_Data/ukb49508.tab'
SECOND_UKBB_FILE = '../glaucoma_project/UKBB_Data_Basket3/ukb51745.tab'
MAIN_HTML_FILE = '../glaucoma_project/UKBB_Data/ukb49508.html'
SECOND_HTML_FILE = '../glaucoma_project/UKBB_Data_Basket3/ukb51745.html'
TOTAL_POP = 502419
TEST_SET_RANDOM_SPLIT = 42
CHUNK_SIZE = 10000

In [2]:
df_selected_genes_imputed = pd.read_pickle(UKBB_PATH+'processed_data/imputed_genes.pkl')

In [3]:
glaucoma_diagnosis = pd.read_pickle(UKBB_PATH+'processed_data/glaucoma_diagnosis_for_genetic_ml.pkl')

In [4]:
glaucoma_diagnosis_incl_no_genetic = pd.read_pickle(UKBB_PATH+'processed_data/glaucoma_diagnosis.pkl')

In [5]:
imputed_matrix = pd.read_pickle(UKBB_PATH+'processed_data/imputed_matrix_selected_clinical_features.pkl')

Get test-train split data

In [6]:
train_idx, test_idx = train_test_split(range(TOTAL_POP),random_state=TEST_SET_RANDOM_SPLIT, test_size=0.1)

In [7]:
def get_dataframe_for_analysis(column_names, idx=train_idx):
    print('Preparing dataframe...')
    
    cols_in_main_file = pd.read_csv(MAIN_UKBB_FILE, sep='\t', nrows=1).columns
    cols_in_second_file = pd.read_csv(SECOND_UKBB_FILE, sep='\t', nrows=1).columns
    column_names_main = [col for col in column_names if col in cols_in_main_file]
    column_names_second = [col for col in column_names if (not col in cols_in_main_file) and (col in cols_in_second_file)]
    
    
    df_selected_cols = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(MAIN_UKBB_FILE, sep='\t', chunksize=CHUNK_SIZE, usecols=column_names_main+['f.eid']), total=TOTAL_POP // CHUNK_SIZE):
        df_selected_cols = pd.concat([df_selected_cols, chunk])
        
    df_selected_cols_second = pd.DataFrame()
    for chunk in tqdm(pd.read_csv(SECOND_UKBB_FILE, sep='\t', chunksize=CHUNK_SIZE, usecols=column_names_second+['f.eid']), total=TOTAL_POP // CHUNK_SIZE):
        df_selected_cols_second = pd.concat([df_selected_cols_second, chunk])
    
    return pd.merge(df_selected_cols,df_selected_cols_second,how='left',on='f.eid').iloc[idx].reset_index(drop=True)

In [11]:
df_glaucoma_data = pd.read_pickle(UKBB_PATH+'processed_data/Clinical Feature Selection/glaucoma_data_checkpoint.pkl')

In [12]:
len(df_glaucoma_data)

452177

TESTING--

In [3]:
example_model_1a = load('../glaucoma_project/Models/LogisticRegression_clinical_1a_model.joblib')

In [6]:
pred_1a = example_model_1a.predict_proba(ultimate_merged_df[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']])

In [13]:
evaluate_model(ultimate_merged_df['Total Glaucoma'], pred_1a)

AUC 0.7065832969203975
Youden's J threshold 0.4787914537612834
Accuracy 0.5555936467051027
Sensitivity 0.7581982661138333
Specificity 0.5530725339874815


(0.7065832969203975,
 0.4787914537612834,
 0.5555936467051027,
 0.7581982661138333,
 0.5530725339874815)

-- TESTING

In [13]:
imputed_matrix

,Waist circumference_0,Townsend deprivation index at recruitment,Number of days/week of vigorous physical activity 10+ minutes_0,Time spend outdoors in summer_0,Time spent outdoors in winter_0,Time spent watching television (TV)_0,Time spent using computer_0,Cooked vegetable intake_0,Fresh fruit intake_0,Dried fruit intake_0,...,On Ibuprofen,Sports club or gym,Religious group,Swimming/cycling/bowling/keep fit,On Fish Oil,On Glucosamine,Family history diabetes,Family history hypertension,IBS Diagnosis,Hiatus Hernia Diagnosis
0,-0.911800,-1.118830,-4.412120e-01,0.929051,-1.454543e-01,1.950884,0.000000,-0.942502,-0.818550,-5.055417e-01,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,-1.209223,0.084647,-4.412120e-01,0.478434,-7.553898e-01,-0.573127,-0.178616,-0.412012,-0.184510,-5.055417e-01,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.541941,-1.556288,-4.412120e-01,-0.422799,-1.454543e-01,-0.573127,-0.178616,-0.942502,-0.818550,6.851648e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.021451,0.072867,-9.660884e-01,-1.324032,-2.708658e-16,-0.573127,-0.178616,-0.412012,0.449530,-5.055417e-01,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.019530,-0.901135,-4.412120e-01,-0.422799,-7.553898e-01,1.319881,0.000000,0.118478,11.228209,5.235040e+00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445154,1.095807,-0.567292,-4.412120e-01,-0.422799,4.644813e-01,-1.204130,0.000000,0.648969,-0.818550,-6.373327e-17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
445155,-1.357934,0.955716,-9.660884e-01,-0.422799,-7.553898e-01,-1.204130,-0.178616,0.648969,-0.818550,6.851648e-02,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
445156,0.798383,-0.769106,-2.330919e-16,0.000000,-2.708658e-16,1.319881,-0.178616,-0.412012,1.083570,-6.373327e-17,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
445157,0.129181,-0.993041,8.366436e-02,-0.873415,-7.553898e-01,0.057876,0.000000,-0.412012,-0.184510,-5.055417e-01,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [14]:
eids = pd.read_pickle(UKBB_PATH+'processed_data/eid_list_clinical_models.pkl')

In [25]:
imputed_matrix['f.eid'] = eids.reset_index(drop=True)

In [17]:
imputed_matrix['Total Glaucoma'] = glaucoma_diagnosis_incl_no_genetic

# Make training set for ensemble methods - both has genetic data and excluding pre-2010 glaucoma cases

In [30]:
ultimate_merged_df = pd.merge(imputed_matrix,df_selected_genes_imputed,how='inner',on='f.eid')

In [31]:
glaucoma_diagnosis.value_counts()

0    426407
1     12098
Name: Total Glaucoma, dtype: int64

In [32]:
imputed_matrix['Total Glaucoma'].value_counts()

0    439661
1      5498
Name: Total Glaucoma, dtype: int64

In [34]:
ultimate_merged_df['Total Glaucoma'].value_counts()

0    426407
1      5306
Name: Total Glaucoma, dtype: int64

In [35]:
ultimate_merged_df.to_pickle(UKBB_PATH+'processed_data/merged_df_for_ensemble_training.pkl')

In [7]:
ultimate_merged_df = pd.read_pickle(UKBB_PATH+'processed_data/merged_df_for_ensemble_training.pkl')

# Getting classifiers

Going to use a logistic regression meta-classifier to combine predictions

In [64]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier

50% for training the level 1 models, 50% for training the metaclassifier

In [9]:
train_idx, test_idx = train_test_split(range(len(ultimate_merged_df)),random_state=43, test_size=0.5)

In [7]:
len(train_idx)

215856

Define the best models

In [16]:
model_1a = LogisticRegression(C=0.001, class_weight='balanced', max_iter=300,
                   solver='liblinear')
model_1b = GradientBoostingClassifier(max_depth=1, n_estimators=1000, random_state=42)
model_2 = GradientBoostingClassifier(max_depth=1, n_estimators=1000, random_state=42)
model_3 = GradientBoostingClassifier(max_depth=1, n_estimators=1000, random_state=42)

In [84]:
model_g = MLPClassifier(random_state=42,alpha=0.01,hidden_layer_sizes=(100,))

In [10]:
classification_dict = load(UKBB_PATH+'processed_data/classification_dict.joblib')

In [11]:
train_data = ultimate_merged_df.iloc[train_idx]
val_data = ultimate_merged_df.iloc[test_idx]

In [12]:
y_train = train_data['Total Glaucoma']
y_val = val_data['Total Glaucoma']

In [92]:
X_train_1a = train_data[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']]
X_train_1b = train_data[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b']]]
X_train_2 = train_data[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b','2']]]
X_train_3 = train_data[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b','2','3']]]
X_train_G = train_data[train_data.columns[245:]]

In [93]:
X_val_1a = val_data[[col for col in classification_dict.keys() if classification_dict[col][1] == '1a']]
X_val_1b = val_data[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b']]]
X_val_2 = val_data[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b','2']]]
X_val_3 = val_data[[col for col in classification_dict.keys() if classification_dict[col][1] in ['1a','1b','2','3']]]
X_val_G = val_data[train_data.columns[245:]]

Training

In [15]:
t0 = time.time()
model_1a.fit(X_train_1a,y_train)
print('Training took',time.time()-t0)

Training took 1.040104627609253


In [94]:
y_pred_1a = model_1a.predict_proba(X_val_1a)
evaluate_model(y_val, y_pred_1a)

AUC 0.7043875602563623
Youden's J threshold 0.4818424999123913
Accuracy 0.5593656911751762
Sensitivity 0.758266818700114
Specificity 0.5569114460713046


(0.7043875602563623,
 0.4818424999123913,
 0.5593656911751762,
 0.758266818700114,
 0.5569114460713046)

In [25]:
dump(model_1a, '../glaucoma_project/Models/model_1a_50%.joblib')

['../glaucoma_project/Models/model_1a_50%.joblib']

In [26]:
t0 = time.time()
model_1b.fit(X_train_1b,y_train)
print('Training took',time.time()-t0)

Training took 396.88813376426697


In [95]:
y_pred_1b = model_1b.predict_proba(X_val_1b)
evaluate_model(y_val, y_pred_1b)

AUC 0.7042598663068091
Youden's J threshold 0.01264703581337873
Accuracy 0.6045669123540122
Sensitivity 0.7134169517293805
Specificity 0.6032238094791442


(0.7042598663068091,
 0.01264703581337873,
 0.6045669123540122,
 0.7134169517293805,
 0.6032238094791442)

In [28]:
dump(model_1b, '../glaucoma_project/Models/model_1b_50%.joblib')

['../glaucoma_project/Models/model_1b_50%.joblib']

In [29]:
t0 = time.time()
model_2.fit(X_train_2,y_train)
print('Training took',time.time()-t0)

Training took 426.6947555541992


In [96]:
y_pred_2 = model_2.predict_proba(X_val_2)
evaluate_model(y_val, y_pred_2)

AUC 0.734104492773896
Youden's J threshold 0.013376669800552575
Accuracy 0.6535020870298393
Sensitivity 0.7020144431774991
Specificity 0.6529034920694474


(0.734104492773896,
 0.013376669800552575,
 0.6535020870298393,
 0.7020144431774991,
 0.6529034920694474)

In [58]:
# evaluate only on those with data available
y_pred_2 = model_2.predict_proba(X_val_2.reset_index(drop=True)[had_ophthal_assessment])
evaluate_model(y_val.reset_index(drop=True)[had_ophthal_assessment], y_pred_2)

AUC 0.7949885728733174
Youden's J threshold 0.0194972044205351
Accuracy 0.7040319617927994
Sensitivity 0.7439613526570048
Specificity 0.7032581218986986


(0.7949885728733174,
 0.0194972044205351,
 0.7040319617927994,
 0.7439613526570048,
 0.7032581218986986)

In [31]:
dump(model_2, '../glaucoma_project/Models/model_2_50%.joblib')

['../glaucoma_project/Models/model_2_50%.joblib']

In [32]:
t0 = time.time()
model_3.fit(X_train_3,y_train)
print('Training took',time.time()-t0)

Training took 755.641188621521


In [97]:
y_pred_3 = model_3.predict_proba(X_val_3)
evaluate_model(y_val, y_pred_3)

AUC 0.7358864326419248
Youden's J threshold 0.012255301051084681
Accuracy 0.6185530235294663
Sensitivity 0.7347016343595592
Specificity 0.6171198634312889


(0.7358864326419248,
 0.012255301051084681,
 0.6185530235294663,
 0.7347016343595592,
 0.6171198634312889)

In [59]:
# evaluate only on those with data available
y_pred_3 = model_3.predict_proba(X_val_3.reset_index(drop=True)[had_ophthal_assessment])
evaluate_model(y_val.reset_index(drop=True)[had_ophthal_assessment], y_pred_3)

AUC 0.7963840030176841
Youden's J threshold 0.017488456339208855
Accuracy 0.660268185157972
Sensitivity 0.7898550724637681
Specificity 0.6577567643479075


(0.7963840030176841,
 0.017488456339208855,
 0.660268185157972,
 0.7898550724637681,
 0.6577567643479075)

In [34]:
dump(model_3, '../glaucoma_project/Models/model_3_50%.joblib')

['../glaucoma_project/Models/model_3_50%.joblib']

In [85]:
t0 = time.time()
model_g.fit(X_train_G,y_train)
print('Training took',time.time()-t0)

Training took 328.431099653244


In [98]:
y_pred_g = model_g.predict_proba(X_val_G)
evaluate_model(y_val, y_pred_g)

AUC 0.6266759131232371
Youden's J threshold 0.008161650320730641
Accuracy 0.5799765585549692
Sensitivity 0.6092740402888636
Specificity 0.5796150563252136


(0.6266759131232371,
 0.008161650320730641,
 0.5799765585549692,
 0.6092740402888636,
 0.5796150563252136)

In [88]:
dump(model_g, '../glaucoma_project/Models/model_g_50%.joblib')

['../glaucoma_project/Models/model_g_50%.joblib']

In [89]:
model_1a = load('../glaucoma_project/Models/model_1a_50%.joblib')
model_1b = load('../glaucoma_project/Models/model_1b_50%.joblib')
model_2 = load('../glaucoma_project/Models/model_2_50%.joblib')
model_3 = load('../glaucoma_project/Models/model_3_50%.joblib')
model_g = load('../glaucoma_project/Models/model_g_50%.joblib')

In [99]:
df_proba = pd.DataFrame()
df_proba['1A'] = y_pred_1a[:,1]
df_proba['1B'] = y_pred_1b[:,1]
df_proba['2A'] = y_pred_2[:,1]
df_proba['3A'] = y_pred_3[:,1]
df_proba['G'] = y_pred_g[:,1]

In [102]:
had_ophthal_assessment = ~val_data['Intra-ocular pressure, corneal-compensated (highest)'].apply(lambda x: x < 2e-15 and x > -2e15).reset_index(drop=True)

# Model Evaluation

In [103]:
from sklearn.metrics import roc_curve, accuracy_score, confusion_matrix

In [104]:
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    return specificity

In [105]:
def sensitivity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp+fn)
    return sensitivity

In [106]:
def cutoff_youdens_j(fpr,tpr,thresholds):
    j_scores = tpr-fpr
    j_ordered = sorted(zip(j_scores,thresholds))
    return j_ordered[-1][1]

In [107]:
def evaluate_model(y_true, y_pred_proba):
    if len(y_pred_proba.shape) == 2:
        probs = y_pred_proba[:,1]
    else:
        probs = y_pred_proba
    auc = roc_auc_score(y_true,probs)
    print('AUC',auc)
    fpr, tpr, thresholds = roc_curve(y_true, probs)
    yj_threshold = cutoff_youdens_j(fpr, tpr, thresholds)
    print('Youden\'s J threshold',yj_threshold)
    acc = accuracy_score(y_true,(probs > yj_threshold).astype(int))
    print('Accuracy',acc)
    sens = sensitivity(y_true,(probs > yj_threshold).astype(int))
    print('Sensitivity',sens)
    spec = specificity(y_true,(probs > yj_threshold).astype(int))
    print('Specificity',spec)
    return auc,yj_threshold,acc,sens,spec

# Constructing metaclassifiers

## 1C

In [108]:
meta_classifier_1c = LogisticRegression(class_weight='balanced')

In [109]:
meta_classifier_1c.fit(df_proba[['1B','G']],y_val)

LogisticRegression(class_weight='balanced')

In [110]:
meta_classifier_1c.coef_

array([[57.15218293, 44.83243462]])

In [111]:
y_pred = meta_classifier_1c.predict_proba(df_proba[['1B','G']])

In [112]:
evaluate_model(y_val,y_pred)

AUC 0.7312911064365575
Youden's J threshold 0.46365234271117983
Accuracy 0.6284901578359747
Sensitivity 0.7198783732421132
Specificity 0.627362516766248


(0.7312911064365575,
 0.46365234271117983,
 0.6284901578359747,
 0.7198783732421132,
 0.627362516766248)

In [113]:
dump(meta_classifier_1c, '../glaucoma_project/Models/meta_classifier_1c.joblib')

['../glaucoma_project/Models/meta_classifier_1c.joblib']

## For 2B and 3B, limit the set to those that have had an ophthal assessment!

In [114]:
df_proba_had_ophthal_assessment = df_proba.reset_index(drop=True)[had_ophthal_assessment].reset_index(drop=True)

In [115]:
y_val_had_ophthal_assessment = y_val.reset_index(drop=True)[had_ophthal_assessment].reset_index(drop=True)

## 2B

In [116]:
meta_classifier_2b = LogisticRegression(class_weight='balanced')

In [117]:
meta_classifier_2b.fit(df_proba_had_ophthal_assessment[['2A','G']],y_val_had_ophthal_assessment)

LogisticRegression(class_weight='balanced')

In [118]:
meta_classifier_2b.coef_

array([[23.72579044,  7.967164  ]])

In [119]:
y_pred = meta_classifier_2b.predict_proba(df_proba_had_ophthal_assessment[['2A','G']])

In [120]:
evaluate_model(y_val_had_ophthal_assessment,y_pred)

AUC 0.8027645821941258
Youden's J threshold 0.4228528916581882
Accuracy 0.7054096252755327
Sensitivity 0.7463768115942029
Specificity 0.7046156726898231


(0.8027645821941258,
 0.4228528916581882,
 0.7054096252755327,
 0.7463768115942029,
 0.7046156726898231)

In [121]:
dump(meta_classifier_2b, '../glaucoma_project/Models/meta_classifier_2b.joblib')

['../glaucoma_project/Models/meta_classifier_2b.joblib']

## 3B

In [122]:
meta_classifier_3b = LogisticRegression(class_weight='balanced')

In [123]:
meta_classifier_3b.fit(df_proba_had_ophthal_assessment[['3A','G']],y_val_had_ophthal_assessment)

LogisticRegression(class_weight='balanced')

In [124]:
y_val_had_ophthal_assessment

0        0
1        0
2        0
3        0
4        0
        ..
21771    0
21772    0
21773    0
21774    0
21775    0
Name: Total Glaucoma, Length: 21776, dtype: int64

In [125]:
meta_classifier_3b.coef_

array([[24.08228414,  8.16548987]])

In [126]:
y_pred = meta_classifier_3b.predict_proba(df_proba_had_ophthal_assessment[['3A','G']])

In [127]:
evaluate_model(y_val_had_ophthal_assessment,y_pred)

AUC 0.8044328567545331
Youden's J threshold 0.42039642450049464
Accuracy 0.6997152828802351
Sensitivity 0.7560386473429952
Specificity 0.6986237243703773


(0.8044328567545331,
 0.42039642450049464,
 0.6997152828802351,
 0.7560386473429952,
 0.6986237243703773)

In [128]:
dump(meta_classifier_3b, '../glaucoma_project/Models/meta_classifier_3b.joblib')

['../glaucoma_project/Models/meta_classifier_3b.joblib']